In [1]:
#%%
import pathpy as pp
import random

: 

In [2]:

n = pp.Network(directed=False)
clusters = {}
colors = {}
for i in range(30):
    n.add_node(str(i))
    n.nodes[str(i)]['cluster'] = str(int(i/10))
    if str(int(i/10)) == '0':
        colors[str(i)] = 'red'
    elif str(int(i/10)) == '1':
        colors[str(i)] = 'blue'
    else: 
        colors[str(i)] = 'green'
    clusters[str(i)] = str(int(i/10))

In [ ]:
# for v in n.nodes:
#     print('{0} {1}'.format(v, n.nodes[v]['cluster']))

#%% k-regular topology
k = 30

for i in n.nodes:
    for j in n.nodes:
        if i!=j:
            n.add_edge(i, j)
#stubs = []
#for v in n.nodes:
#    for i in range(0, k):
#        stubs.append(v)

pp.visualisation.plot(n, node_color=colors)

In [4]:
#%% generate random temporal network
t = pp.TemporalNetwork()
time = 0
for i in range(10000):
    u = random.sample(list(n.nodes.keys()), 1)[0]
    v = random.sample(n.successors[u], 1)[0]
    w = random.sample(n.successors[v], 1)[0]
    t.add_edge(u, v, time)
    t.add_edge(v, w, time+1)
    time += 2

#print(t)

In [ ]:
# for each center node v of a path ... 
# 1.) find two edges (u, v, t1) (v, w, t1+1) with u, v in cluster 1 and w in cluster 2
# 2.) find two edges (x, v, t2) (v, z, t2+1) with x in cluster 2 and v, z in cluster 1
# 3.) swap timestamps to (u, v, t1), (v, z, t1+1) and (x, v, t2), (v, w, t2+1)
# we have changed two paths of length two that connect nodes in different clusters 
# to paths that connect nodes in the same cluster, without changing link statistics!
candidates = {}
from collections import defaultdict
candidates['0-1'] = defaultdict(list)
candidates['1-0'] = defaultdict(list)

candidates['0-2'] = defaultdict(list)
candidates['2-0'] = defaultdict(list)

candidates['2-1'] = defaultdict(list)
candidates['1-2'] = defaultdict(list)

others = []
i = 1
while i < len(t.ordered_times):
    prev = t.ordered_times[i-1]
    time = t.ordered_times[i]
    (u, v, t1) = t.time[prev][0]
    (v, w, t2) = t.time[time][0]
    # each edge is in exactly one of the lists
    
    assert t.time[prev][0][1] == t.time[time][0][0], 'Error!'

    if clusters[u] == clusters[v] and clusters[v] != clusters[w]:
        candidates[clusters[u]+'-'+clusters[w]][v].append( (t1, t2) )
    elif clusters[u] != clusters[v] and clusters[v] == clusters[w]:
        candidates[clusters[u]+'-'+clusters[w]][v].append( (t1, t2) )
        # print('{0} -> {1} -> {2}'.format(u,v,w))
    else:
        others.append( (t1, t2) )
    i += 2

# swap timestamps of edges for the following pairs of candidates
swaps = [('0-1', '1-0'), # from 0->1 and 1->0 to 0->0 and 1->1
         ('0-2', '2-0'),
         ('1-2', '2-1')]

t_clustered = pp.TemporalNetwork()
swap_c = 0
for s in swaps:
    for v in candidates[s[0]]:
        while candidates[s[0]][v] and candidates[s[1]][v]:
            # found four candidate edges for time-stamp swap
            (t1, t2) = candidates[s[0]][v].pop()
            (u, v, t1) = t.time[t1][0]
            (v, w, t2) = t.time[t2][0]

            (t3, t4) = candidates[s[1]][v].pop()
            (x, v, t3) = t.time[t3][0]
            (v, z, t4) = t.time[t4][0]

            t_clustered.add_edge(u, v, t1)
            t_clustered.add_edge(v, z, t2)
            t_clustered.add_edge(x, v, t3)
            t_clustered.add_edge(v, w, t4)            
            swap_c +=1
    # add all remaining edges
        while candidates[s[0]][v]:
            (t1, t2) = candidates[s[0]][v].pop()
            (source, target, time) = t.time[t1][0]
            t_clustered.add_edge(source, target, time)
            (source, target, time) = t.time[t2][0]
            t_clustered.add_edge(source, target, time)
        while candidates[s[1]][v]:
            (t1, t2) = candidates[s[1]][v].pop()
            (source, target, time) = t.time[t1][0]
            t_clustered.add_edge(source, target, time)
            (source, target, time) = t.time[t2][0]
            t_clustered.add_edge(source, target, time)

while others:
    (t1, t2) = others.pop()
    (source, target, time) = t.time[t1][0]
    t_clustered.add_edge(source, target, time)
    (source, target, time) = t.time[t2][0]
    t_clustered.add_edge(source, target, time)
print('Swapped {0} edges'.format(swap_c))
print(t_clustered)


In [6]:
#%%
#t_clustered.write_file(filename='temporal_clusters_real_kregular.tedges')

In [ ]:
#%%
p = pp.path_extraction.paths_from_temporal_network_dag(t_clustered)
print(p)
#p.write_file('temporal_clusters_real_kregular.ngram')
mog = pp.MultiOrderModel(p, max_order=3)
print(mog.estimate_order())

With my impl

In [8]:
from fruchtermanReingold import FruchtermanReingold
from projector import Projector

In [9]:
from matplotlib.collections import LineCollection
import matplotlib.pyplot as plt
def plot_network(network, nodes_position, figsize=[20,20]):
    """Functiton that simply plots a network in matplotlib"""
    mp = network.node_to_name_map()

    points = nodes_position
    edges = [(mp[e[0]],mp[e[1]]) for e in network.edges]

    #adding edges
    lc = LineCollection(points[edges], linewidths=2.5)
    fig = plt.figure(figsize=figsize) 
    plt.gca().add_collection(lc)

    plt.axis('off')
    plt.xlim(points[:,0].min(), points[:,0].max())
    plt.ylim(points[:,1].min(), points[:,1].max())

    #colors = plt.cm.rainbow(np.linspace(0, 1, len(points)))

    plt.scatter(points[:,0], points[:,1], s= 80, zorder = 2)#
#     fig.savefig('time_aware_figure{}.png'.format(int(time.time() // 1)),quality = 95)

In [ ]:
order = 2
paths = p
#observed_paths, sub_paths = number_longest_and_subpaths(paths)
#alpha  = [1/(sub_paths[i]/sub_paths[1]) for i in range(1,order+1)]
alpha = [.3,1,0,0]
first_order_network = pp.Network.from_paths(paths)
super_network_mon = pp.MultiOrderModel(paths = paths, max_order = order)

projected_network_mon = Projector(super_network_mon,alpha=alpha).get_projection()#_unweighted()

FR_mon = FruchtermanReingold(projected_network_mon,repulsion_multiplier=10)
list_positions_mon = FR_mon.run(10000)
dict_positions_mon = FR_mon.node_to_pos_map 


In [ ]:
plot_network(first_order_network, list_positions_mon)